In [2]:
!kaggle competitions download -c dogs-vs-cats

In [3]:
# extracting compressed files
from zipfile import ZipFile
dataset = 'dogs-vs-cats.zip'
with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print("The Dataset is Extracted")

In [4]:
# extracting train folder cause this is the only one we want
dataset = 'train.zip'
with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print("The Dataset is Extracted")

In [5]:
# #counting number of files in the train folder
import os 
paths,dirs,files=next(os.walk('train'))
file_count=len(files)
print(f"{file_count} Number of images found")

## Prepocessing the images for the model

In [6]:
#Importing the dependancies 
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

### displaying images

In [7]:
# #displaying the images of dog
img = mpimg.imread('train/dog.8928.jpg')
plt.imshow(img)

In [8]:
# #displaying the images of cat
img2 = mpimg.imread('train/cat.4928.jpg')
plt.imshow(img2)

##### These images are not in same shape and since our model requires a specific shape. we manipulate and resize all images in the required size

In [9]:
file_names=os.listdir('train')
file_names

In [10]:
dog_count=0
cat_count=0

for img_file in file_names:
    name=img_file[0:3]
    if name == 'dog':
        dog_count+=1
    elif name == 'cat':
        cat_count+=1
print("Number of dogs =" , dog_count)
print("Number of cats =" , cat_count)
        
    

In [11]:
os.mkdir('dogs')
os.mkdir('cats')

In [12]:
#splitting dogs and cats in different folders to make it easier for an even split of datasets
dog_folder = 'dogs/'
cat_folder = 'cats/'
original_folder='train/'

for i in range(25000): #25000 because i know there are 25000 images in the folder
    filename=os.listdir('train')[i]
    img_path=original_folder+filename
    img = Image.open(img_path)
    name=filename[0:3]
    cat_img_path = cat_folder+filename
    dog_img_path = dog_folder+filename
    img = Image.open(img_path)
    if name=='dog':
        img.save(dog_img_path)
        
        

#### Resizing all images

In [13]:
#creating a new directory for resized images

os.mkdir('resizedimages')

#### WE need only 2000 images of cats and 2000 images of dogs since we are using a pre-trained model

In [14]:
#Resizing cat images
original_folder='cats/'
resized_folder='resizedimages/'

for i in range(1000):
    filename=os.listdir('cats')[i]
    img_path=original_folder+filename
    
    img = Image.open(img_path)
    img = img.resize((224,224)) #we need specific size for our mobilenet pretrained model
    img=img.convert('RGB')
    
    new_img_path = resized_folder+filename
    img.save(new_img_path)
    

In [15]:
#Resizing dog images
original_folder='dogs/'
resized_folder='resizedimages/'

for i in range(1000):
    filename=os.listdir('dogs')[i]
    img_path=original_folder+filename
    
    img = Image.open(img_path)
    img = img.resize((224,224)) #we need specific size for our mobilenet pretrained model
    img=img.convert('RGB')
    
    new_img_path = resized_folder+filename
    img.save(new_img_path)
    

In [16]:
# #displaying resized images of dog
img = mpimg.imread('resizedimages/dog.307.jpg')
plt.imshow(img)

In [17]:
# #displaying resized images of cat
img = mpimg.imread('resizedimages/cat.1060.jpg')
plt.imshow(img)

## Creating labels for resized images of dogs and cats
### Cats --> 1
### Dogs --> 0

In [18]:
import os
filename=os.listdir('resizedimages/')
labels=[]
for i in range(2000):
    file_name=filename[i]
    label=file_name[0:3]
    
    if label=='cat':
        labels.append(1)
    elif label=='dog':
        labels.append(0)
            

In [19]:
values,counts =np.unique(labels,return_counts=True)


In [20]:
values

array([0, 1])

In [21]:
counts

array([1000, 1000], dtype=int64)

In [22]:
#Converting all resized images to NumPy arrays

In [23]:
import cv2
import glob


In [24]:
imagedir='resizedimages/'
image_extension= ['png','jpg']

files=[]
[files.extend(glob.glob(imagedir + '*.' + e)) for e in image_extension]

dog_cat_images=np.asarray([cv2.imread(file) for file in files])

In [25]:
dog_cat_images.shape , type(dog_cat_images)

((2000, 224, 224, 3), numpy.ndarray)

In [26]:
features = dog_cat_images
labels = np.asarray(labels)

## splitting the data into train-test sets

In [27]:
features_train , features_test , labels_train , labels_test = train_test_split(features,labels,test_size=0.2,random_state=42,stratify=labels)

In [28]:
features_train.shape , features_test.shape

((1600, 224, 224, 3), (400, 224, 224, 3))

In [29]:
labels_train.shape , labels_test.shape

((1600,), (400,))

In [30]:
# We have 1600 training datasets and 400 test datasets


### Feature scaling 

In [31]:
#Dividing all values by 255 because the that is my maximum intensity of colors
scaled_features_train = features_train/255
scaled_features_test = features_test/255

In [32]:
scaled_features_train

array([[[[0.35294118, 0.50588235, 0.63137255],
         [0.36862745, 0.52156863, 0.64705882],
         [0.38039216, 0.53333333, 0.65882353],
         ...,
         [0.23921569, 0.34117647, 0.40784314],
         [0.23921569, 0.34117647, 0.40784314],
         [0.22745098, 0.32941176, 0.39607843]],

        [[0.37647059, 0.52941176, 0.65490196],
         [0.39215686, 0.54509804, 0.67058824],
         [0.4       , 0.55294118, 0.67843137],
         ...,
         [0.22352941, 0.3254902 , 0.39215686],
         [0.21568627, 0.31764706, 0.38431373],
         [0.2       , 0.30196078, 0.36862745]],

        [[0.38431373, 0.5372549 , 0.6627451 ],
         [0.4       , 0.55294118, 0.67843137],
         [0.41176471, 0.56470588, 0.69019608],
         ...,
         [0.21568627, 0.30980392, 0.38039216],
         [0.20392157, 0.29803922, 0.36862745],
         [0.18431373, 0.27843137, 0.34901961]],

        ...,

        [[0.14117647, 0.24705882, 0.38039216],
         [0.15294118, 0.25882353, 0.39215686]

## Building the Neural Network
### Using a Pre trained model

In [33]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

In [34]:
#Uploading the mobilenet model from the url
MobileNet_model='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model=hub.KerasLayer(MobileNet_model,input_shape=(224,224,3),trainable=False)

In [35]:
num_of_classes=2
model=tf.keras.Sequential([pretrained_model,
                          tf.keras.layers.Dense(num_of_classes)])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [36]:
#compiling the model with preferred arguments
model.compile(optimizer='adam',
             loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['acc']
             )

In [37]:
model.fit(scaled_features_train,labels_train,epochs=5)

Epoch 1/5
50/50 [==============================] - 247s 4s/step - loss: 0.2054 - acc: 0.9175
Epoch 2/5
50/50 [==============================] - 326s 7s/step - loss: 0.0794 - acc: 0.9706
Epoch 3/5
50/50 [==============================] - 397s 8s/step - loss: 0.0587 - acc: 0.9781
Epoch 4/5
50/50 [==============================] - 419s 8s/step - loss: 0.0458 - acc: 0.9850
Epoch 5/5
50/50 [==============================] - 383s 8s/step - loss: 0.0360 - acc: 0.9900


In [38]:
# Evaluating the model on the test data
score, acc = model.evaluate(scaled_features_test,labels_test)
print('Test Loss=',score)
print('Test Accuracy=',acc)

13/13 [==============================] - 125s 9s/step - loss: 0.0665 - acc: 0.9700
Test Loss= 0.06651473045349121
Test Accuracy= 0.9700000286102295


In [39]:
# With an accuracy of 97% , Basically we can say that the model can identify 97 out of 100 pictures correctly


### Defining a Predictor Function 

In [40]:
def Cat_Or_Dog(image_path):
    input_image=cv2.imread(image_path)
    cv2.imshow("test",input_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    input_image_resize=cv2.resize(input_image,(224,224))
    input_image_scaled=input_image_resize/255
    image_reshaped = input_image_scaled.reshape(1,224,224,3) # 1 indicates that we are maling predictions for 1 image
    prediction=model.predict(image_reshaped)
    print(prediction)
    pred_label = np.argmax(prediction)
    print(pred_label)
    if pred_label==0:
        print("Its a Dog.. Woof!")
    else:
        print("Meowww..Its a Cat")
        



In [41]:
Cat_Or_Dog('cat_test.jpg')

1/1 [==============================] - 6s 6s/step
[[-5.0976167  5.704837 ]]
1
Meowww..Its a Cat


In [42]:
Cat_Or_Dog('dog_test.jpg')

1/1 [==============================] - 0s 263ms/step
[[ 3.8217309 -2.2876353]]
0
Its a Dog.. Woof!


In [45]:
tf.saved_model.LoadOptions(
    allow_partial_checkpoint=False,
    experimental_io_device='/job:localhost',
    experimental_skip_checkpoint=False
)

### Saving and Loading my Model

In [47]:
model.save('my_model.h5')

In [49]:
loaded_model=keras.models.load_model('my_model.h5',custom_objects={'KerasLayer':hub.KerasLayer})


In [50]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


### Building a Streamlit UI for the WebApp

In [ ]:
import streamlit 
